# Module 3, Section 2: Interacting with Deployments via SDK

Our agent is now deployed to production with monitoring in place. But how do we actually **use** it?

In this section, we'll learn how to interact with our deployed agent programmatically using the **LangGraph SDK**. This enables you to:
- Build custom UIs and applications
- Automate testing and workflows
- Handle human-in-the-loop interrupts
- Integrate with existing systems

By the end, you'll be able to call your deployed agent from any Python application.


## Setup


In [ ]:
from dotenv import load_dotenv
from langgraph_sdk import get_client

load_dotenv()

# Connect to your deployment
DEPLOYMENT_API_URL = "<your-deployment-api-url>"
client = get_client(url=DEPLOYMENT_URL)

---

## 1. Basic Invocation

Let's start simple - call our deployed agent and get a response.

We'll ask about product information (no identity verification needed).

![NOTE]
First invocation to subagent that uses vector search will build the vector DB on-demand, one time setup.

In [2]:
# Create a thread (conversation session)
thread = await client.threads.create()

# Send a message
input_message = {
    "messages": [
        {
            "role": "user",
            "content": "What are the key features of the Sony WH-1000XM5 headphones?",
        }
    ]
}

# Invoke the agent (wait for completion)
response = await client.runs.wait(
    thread["thread_id"],
    "supervisor_hitl_sql_agent",  # Your deployed graph name
    input=input_message,
)

# Print the final response
final_message = response["messages"][-1]["content"]
print(f"Agent: {final_message}")

Agent: Based on the documentation, here are the **key features of the Sony WH-1000XM5 headphones**:

**Core Features:**
- **Industry-Leading Active Noise Cancellation (ANC)** - Eight microphones with an HD Noise Cancelling Processor QN1 that adjusts 700 times per second
- **Premium 30mm Drivers** - Delivering rich, detailed audio quality
- **30-Hour Battery Life** - Providing extended listening for all-day use
- **Multi-Device Connectivity** - Seamless switching between devices
- **Touch Controls** - Intuitive right earcup controls for play/pause, volume, and track control

**Technical Highlights:**
- **8 Microphones Total** - 4 beamforming microphones for clear calls and 4 ANC microphones for noise cancellation
- **Universal Bluetooth Connectivity** - Works with iOS, Android, Windows, Mac, and all Bluetooth-enabled devices
- **Sony Headphones Connect App** - Customize EQ, adjust ANC settings, and install firmware updates

**Performance Features:**
- Exceptional at blocking low-frequen

**What just happened?**
1. Created a new thread (like starting a new conversation)
2. Sent a message to the agent
3. Waited for the agent to complete
4. Retrieved the final message

**Note:** Check LangSmith - you'll see this trace in your `techhub-workshop-deployment` project!


---

## 2. Human-in-the-Loop (HITL) with SDK

Our agent requires email verification for personal account questions. Let's see how to handle **interrupts** programmatically.


### Step 1: Start a Conversation That Requires Verification


In [25]:
# Create a new thread
thread = await client.threads.create()
thread_id = thread["thread_id"]

# Ask about order status (requires email verification)
input_message = {
    "messages": [{"role": "user", "content": "What's the status of my recent order?"}]
}

# Run the graph until the interrupt is hit.
result = await client.runs.wait(
    thread_id,
    "supervisor_hitl_sql_agent",
    input=input_message,
)

# Print the interrupt
print(result["__interrupt__"])

[{'value': 'Please provide your email:', 'id': '1ec75f612c53e695e24a6877c08ed58f'}]


### Step 2: Check the Interrupt and Get State


In [26]:
# Get the current state
state = await client.threads.get_state(thread_id)

# The agent is waiting for email input
# Check the last message
last_message = state["values"]["messages"][-1]["content"]
print(f"Agent says: {last_message}")

Agent says: To access information about your account or orders, please provide your email address.


### Step 3: Provide Email and Resume


In [ ]:
# Provide the email address
email_input = {
    "messages": [{"role": "user", "content": "My email is consultant.tech@gmail.com"}]
}

# Resume the run
response = await client.runs.wait(
    thread_id,
    "supervisor_hitl_sql_agent",
    input=email_input,
)

# Get the final answer
final_message = response["messages"][-1]["content"]
print(f"\nAgent: {final_message}")


Agent: Great! Here's the status of your recent order:

**Order ID:** ORD-2025-0044  
**Order Date:** October 15, 2025  
**Status:** Processing

Your order is currently being processed and hasn't been shipped yet. It should move to the shipping stage shortly. If you have any other questions about your order or need additional details, feel free to ask!


### Step 4: Ask a Follow-Up (No Interrupt This Time!)


In [29]:
# Ask a follow-up question (customer_id already verified!)
followup_message = {
    "messages": [{"role": "user", "content": "What exactly was in that order?"}]
}

response = await client.runs.wait(
    thread_id,
    "supervisor_hitl_sql_agent",
    input=followup_message,
)

final_message = response["messages"][-1]["content"]
print(f"Agent: {final_message}")

Agent: Here are the details of your order:

**Order ORD-2025-0044**

**Item:**
- **Dell UltraSharp 27" 4K Monitor** (Qty: 1)
- **Price:** $549.00

**Order Total:** $549.00

Your order contains one Dell UltraSharp 27" 4K Monitor and is currently being processed. Once it ships, you'll receive a tracking number. Is there anything else you'd like to know about your order?


**What happened?**
1. First question triggered email verification (interrupt)
2. We provided the email and the agent resumed
3. Follow-up question didn't interrupt - `customer_id` is already in state!

This is how you build custom UIs with HITL flows.

**Learn more:** [HITL with SDK Docs](https://docs.langchain.com/langsmith/add-human-in-the-loop)


---

## 3. Monitoring Production Traces

Every SDK call creates a trace in LangSmith. Let's verify our online evaluator is working.


### Check LangSmith for Your Traces

**Steps:**
1. Go to **LangSmith** → **Projects** → **techhub-workshop-deployment**
2. You'll see all the traces from your SDK calls
3. Click on any trace to see:
   - Full conversation history
   - Tool calls and reasoning
   - Latency metrics
   - Any errors or warnings

**After 1 minute** (your thread idle time), check the **Feedback** tab on each trace:
- You should see `user_sentiment: positive` (since we had helpful conversations)
- The online evaluator runs automatically!
